In [ ]:
import json
from utils import Automata # Assuming Automata class is in utils.py

def create_phone_automata():
    # 1. Define States
    # q0 to q10 (valid steps), plus q_err (invalid state)
    states = {f"q{i}" for i in range(11)} | {"q_err"}
    
    # 2. Define Alphabet (Digits 0-9)
    alphabet = {str(i) for i in range(10)}
    
    # 3. Define Transitions
    transition = {}
    
    # Helper to add a transition
    def add_trans(src, char, dest):
        if src not in transition: transition[src] = {}
        transition[src][char] = dest

    # -- Logic --
    
    # Step 0: Must start with '0'
    add_trans("q0", "0", "q1")
    for digit in alphabet:
        if digit != "0":
            add_trans("q0", digit, "q_err")

    # Step 1-9: Accept any digit and move to next state
    for i in range(1, 10):
        current = f"q{i}"
        next_s = f"q{i+1}"
        for digit in alphabet:
            add_trans(current, digit, next_s)
            
    # Step 10: Logic for the Accepting State
    # If we receive MORE digits after q10, it's an error (too long)
    for digit in alphabet:
        add_trans("q10", digit, "q_err")
        
    # Error State: Stays in Error
    for digit in alphabet:
        add_trans("q_err", digit, "q_err")

    # 4. Construct the Object
    phone_automata = Automata(
        states=states,
        alphabet=alphabet,
        transition=transition,
        start_state="q0",
        accept_states={"q10"}
    )
    
    return phone_automata

# --- Generate and Test ---

phone_check = create_phone_automata()

# Test cases
print(f"0612345678 (Valid):   {phone_check.generate_trace('0612345678')['s'][-1] in phone_check.accept_states}")
print(f"1234567890 (Bad Start): {phone_check.generate_trace('1234567890')['s'][-1] in phone_check.accept_states}")
print(f"06123 (Too Short):      {phone_check.generate_trace('06123')['s'][-1] in phone_check.accept_states}")
print(f"06123456789 (Too Long): {phone_check.generate_trace('06123456789')['s'][-1] in phone_check.accept_states}")

# --- Export to Schema ---
# This prints the JSON structure you asked for
print("\n--- JSON Schema ---")
print(phone_check.to_json())